## Baseline
This notebook implments a baseline model, which shows you how to handle the data and to provide a first very simple solution to the problem. You may re-use and modify any part of this notebook.

why do we use cnn for nlp
CNN can be used for NLP in orde to extract features from the text. CNNs are good at finding local patterns in the data. In NLP, a word can be represented by its surrounding words. So, CNNs can be used to find the relationship between the words in a sentence. CNNs can also be used for text classification.

In [1]:
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score

In [2]:
data_dir = '../kaggle_data'

In [3]:
torch.manual_seed(0)

We first start by defining the dataset class which takes as input the path to the data and the mode (`train`, `val`, or `train`). This fits a count vectorizer using the training set, and uses it on the validation and test sets.

In [4]:
class BaselineDataset(Dataset):
    def __init__(self, data_dir, mode, vectorizer=None):
        super(BaselineDataset, self).__init__()
        assert mode in ['train', 'val', 'test']
        self.mode = mode

        # load the data
        self.data = pd.read_csv(os.path.join(data_dir, f'{mode}_x.csv'), index_col=0)
        self.data.fillna('', inplace=True)

        # load the labels if not the test set
        if self.mode != 'test':
            self.label = pd.read_csv(os.path.join(data_dir, f'{mode}_y.csv'))

        # train the vectorizer if train set
        if self.mode == 'train':
            self.vectorizer = CountVectorizer()
            self.vectorizer.fit(self.data.values.flatten().tolist())
        # otherwise use the vectorizer given as arguments (which was trained on the train set)
        else:
            self.vectorizer = vectorizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data.iloc[idx, 0]
        x = self.vectorizer.transform([x]).toarray()
        x = torch.tensor(x).float()
        if self.mode == 'test':
            return x, idx
        else:
            y = torch.tensor([self.label.iloc[idx, -2]])
            return x, y, idx

In [5]:
train_dataset = BaselineDataset(data_dir, 'train')
val_dataset = BaselineDataset(data_dir, 'val', train_dataset.vectorizer)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)

We will define two models, one which will be a simple MLP, and another one which will generate random predictions to use as comparison.

In [7]:
class BaselineClassifier(nn.Module):
    def __init__(self, input_size, hidden_dim):
        super(BaselineClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

In [8]:
class RandomClassifier(nn.Module):
    def __init__(self):
        super(RandomClassifier, self).__init__()

    def forward(self, x):
        x = torch.rand(len(x))
        return x

Let's check the performance of the random classifier on the validation set.

In [9]:
def worst_group_accuracy(prediction, y):
    """
        Compute the worst group accuracy, with the groups being defined by ['male', 'female', 'LGBTQ',
        'christian', 'muslim', 'other_religions', 'black', 'white'] for positive and negative toxicity.
        arguments:
            prediction [pandas.DataFrame]: dataframe with 2 columns (index and pred)
            y [pandas.DataFrame]: dataframe containing the metadata
        returns:
            wga [float]: worst group accuracy
    """
    y.loc[prediction.index, 'pred'] = prediction.pred

    categories = ['male', 'female', 'LGBTQ', 'christian', 'muslim', 'other_religions', 'black', 'white']
    accuracies = []
    for category in categories:
        for label in [0, 1]:
            group = y.loc[y[category] == label]
            group_accuracy = (group['y'] == (group['pred'] > 0.5)).mean()
            accuracies.append(group_accuracy)
    print(accuracies)
    wga = np.min(accuracies)
    return wga

In [10]:
def evaluate_model(model, dataloader, criterion):
    """
        Evaluate the model on a given dataloader.
        argument:
            model [torch.nn.Module]: model to evaluate
            dataloader [torch.utils.data.DataLoader]: dataloader on which to evaluate
            criterion [torch.nn.modules.loss]: desired loss to compute
        returns:
            dataset_loss [float]: computed loss on the dataset
            dataset_metric [float]: computed metric on the dataset
    """
    model.eval()
    losses, predictions, indices = [], [], []
    for x, y, idx in tqdm(dataloader, leave=False):
        with torch.no_grad():
            pred = model(x)
        loss = criterion(pred.squeeze(), y.squeeze().float())
        losses.extend([loss.item()] * len(y))
        predictions.extend(pred.detach().squeeze().tolist())
        indices.extend(idx.tolist())

    pred_df = pd.DataFrame({'index': indices, 'pred': predictions})
    dataset_loss = np.mean(losses)
    dataset_metric = worst_group_accuracy(pred_df, dataloader.dataset.label)
    return dataset_loss, dataset_metric

In [11]:
model = RandomClassifier()
criterion = nn.BCELoss()
metric = F1Score(task='binary')

In [12]:
random_val_loss, random_val_metric = evaluate_model(model, val_dataloader, criterion)
print(f'Random classifier validation loss {random_val_loss:.4f} WGA {random_val_metric:.4f}')

  0%|          | 0/177 [00:00<?, ?it/s]

[0.4978844488432513, 0.48982161594963275, 0.49769655628801956, 0.4926158546936004, 0.4965578757175857, 0.511324639670556, 0.496775781442284, 0.49934065934065935, 0.4982586487114, 0.4720379146919431, 0.4974657193284156, 0.4776876267748479, 0.4974728910126815, 0.4854721549636804, 0.4973176092453856, 0.492849668643181]
Random classifier validation loss 1.0027 WGA 0.4720


Now let's train the MLP baseline classifier.

In [13]:
def train_model(model, optimizer, criterion, dataloader):
    """
        Train a model for one epoch.
        arguments:
            model [torch.nn.Module]: model to evaluate
            oprimizer [torch.optim]: optimizer used for training
            criterion [torch.nn.modules.loss]: desired loss to compute
            dataloader [torch.utils.data.DataLoader]: dataloader used for training
        returns:
            dataset_loss [float]: computed loss on the dataset
            dataset_metric [float]: computed metric on the dataset
    """
    model.train()
    losses, predictions, indices = [], [], []
    for x, y, idx in tqdm(dataloader, leave=False):
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred.squeeze(), y.squeeze().float())
        loss.backward()
        optimizer.step()

        losses.extend([loss.item()] * len(y))
        predictions.extend(pred.detach().squeeze().tolist())
        indices.extend(idx.tolist())

    pred_df = pd.DataFrame({'index': indices, 'pred': predictions})
    dataset_loss = np.mean(losses)
    dataset_metric = worst_group_accuracy(pred_df, y = dataloader.dataset.label)
    return dataset_loss, dataset_metric

In [14]:
input_size = len(train_dataset.vectorizer.get_feature_names_out())
model = BaselineClassifier(input_size, 128)
optimizer = optim.AdamW(model.parameters(), lr=0.05, weight_decay=0.1)

In [15]:
train_loss, train_metric = train_model(model, optimizer, criterion, train_dataloader)
mlp_val_loss, mlp_val_metric = evaluate_model(model, val_dataloader, criterion)
print(f'MLP classifier validation loss {mlp_val_loss:.4f} WGA {mlp_val_metric:.4f}')

  0%|          | 0/1051 [00:00<?, ?it/s]

[0.8684267727857943, 0.8279436430727943, 0.8678488277189275, 0.8388422911378435, 0.868784964967884, 0.7140142517814727, 0.8620470491126703, 0.8811055426733488, 0.8696547019805241, 0.7594954851655439, 0.8649302460246711, 0.8242665036674817, 0.87135238595056, 0.6698666127728375, 0.8745898391059681, 0.7030183255479698]


  0%|          | 0/177 [00:00<?, ?it/s]

[0.9067178027959916, 0.8629590766002099, 0.9059533202677594, 0.8764216601595655, 0.9062964572421838, 0.7762525737817433, 0.9005660841742554, 0.9158241758241759, 0.9078709078244718, 0.7843601895734598, 0.903335294383853, 0.8468559837728195, 0.909506524535931, 0.7070217917675545, 0.913336326897171, 0.7363097314265783]
MLP classifier validation loss 0.1999 WGA 0.7070


Once we are happy with our results, we want to make a prediction on the test set. Your submission `.csv` file should contain 2 columns:
- ID: with the id of each prediction (do not shuffle to not mix things up)
- pred: the prediction of the model (thresholded or not)

In [16]:
#model = RandomClassifier()
test_dataset = BaselineDataset(data_dir, 'test', train_dataset.vectorizer)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=False)
model.eval()
test_predictions, indices = [], []
for x, idx in tqdm(test_dataloader, leave=False):
    with torch.no_grad():
        pred = (model(x).squeeze() > 0.5).int()
    test_predictions.extend(pred.tolist())
    indices.extend(idx.tolist())

  0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
pred_df = pd.DataFrame({'ID': indices, 'pred': test_predictions})
pred_df.to_csv('../prediction.csv', index=False)